In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    LeakyReLU,
    Add,
    BatchNormalization,
    Dense,
    GlobalAveragePooling2D,
    Dropout
)
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
(images,labels) = ([],[])
folder = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

for (subdirs, dirs, files) in os.walk(folder):
    for subdir in dirs:
        subjectpath = os.path.join(folder, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            labels.append(subdir)
            images.append(np.array(Image.open(path)))
            break
print(len(images),len(labels))

In [ ]:
plt.figure(figsize=(30, 10))

for i in range(15):
    plt.subplot(2,15,i+1)
    plt.imshow(images[i])
    plt.title(labels[i])
    plt.axis('off')
    plt.grid('off')

for i in range(14):
    plt.subplot(2,15,15+i+1)
    plt.imshow(images[15+i])
    plt.title(labels[i+15])
    plt.axis('off')
    plt.grid('off')
    
plt.show()

In [ ]:
train_gen = ImageDataGenerator(
        rotation_range = 20,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        validation_split = 0.2
)


train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

train_data = train_gen.flow_from_directory(
                train_dir,
                target_size = (200,200),
                batch_size = 32,
                class_mode = 'categorical',
                subset = 'training'
)

validation_dir = ''
validation_data = train_gen.flow_from_directory(
                train_dir,
                target_size = (200,200),
                batch_size = 32,
                class_mode = 'categorical',
                subset = 'validation'
)

classes = list(train_data.class_indices)

In [ ]:
def DarknetConv(inputs, filters, kernel_size, strides):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    return x

def DarknetResidual(inputs, filters):
    shortcut = inputs
    x = DarknetConv(inputs, filters//2,kernel_size=(1,1),strides=(1,1))
    x = DarknetConv(x, filters, kernel_size=(3,3),strides=(1,1))
    x = Add()([x, shortcut])
    return x

In [ ]:
inputs = Input(shape=(200,200,3))
x = DarknetConv(inputs, 32, kernel_size=(3,3),strides=(1,1))

x = DarknetConv(x, 64, kernel_size=(3,3), strides=(2,2))

for _ in range(1):
    x = DarknetResidual(x, 64)
    
x = DarknetConv(x, 128, kernel_size=(3,3), strides=(2,2))

for _ in range(2):
    x = DarknetResidual(x, 128)
    
x = DarknetConv(x, 256, kernel_size=(3,3), strides=(2,2))

for _ in range(8):
    x = DarknetResidual(x, 256)
    
x = DarknetConv(x, 512, kernel_size=(3,3), strides=(2,2))

for _ in range(8):
    x = DarknetResidual(x, 512)
    
x = DarknetConv(x, 1024, kernel_size=(3,3), strides=(2,2))

for _ in range(4):
    x = DarknetResidual(x, 1024)
    
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


output = Dense(29,activation='softmax')(x)

darknet = Model(inputs, output)

In [ ]:
darknet.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

#early_stopping = EarlyStopping(
 #       monitor='val_loss',patience=5,
#        min_delta = 0.001
#)
#reduce_lr = ReduceLROnPlateau(
#        monitor='val_loss',patience=5
#)

In [ ]:
darknet.compile(optimizer='adam', loss='categorical_crossentropy',
               metrics=['acc'])

history = darknet.fit_generator(train_data,
                               epochs=10,
                               validation_data=validation_data,
                               callbacks=[early_stopping])

In [ ]:
#darknet.save_weights('asl_darknet.h5')

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['acc', 'val_acc']].plot();

In [ ]:
sample = np.array(Image.open('../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/H_test.jpg'))
sample_fed = np.expand_dims(sample, 0)
pred= darknet.predict(sample_fed)
pred = classes[np.argmax(pred)]
plt.imshow(sample)
plt.title("Actual: H, Predicted: {}".format(pred))
plt.axis('off')
plt.show()

In [ ]:
sample = np.array(Image.open('../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/M_test.jpg'))
sample_fed = np.expand_dims(sample, 0)
pred= darknet.predict(sample_fed)
pred = classes[np.argmax(pred)]
plt.imshow(sample)
plt.title("Actual:M, Predicted: {}".format(pred))
plt.axis('off')
plt.show()